In [30]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import gc

# Mengabaikan peringatan untuk kejelasan output
warnings.filterwarnings('ignore')

In [32]:
# --- 1. Fungsi Pemuatan Data (Load Data) ---
def load_data():
    """Memuat semua file CSV."""
    file_paths = {
        "train": "application_train.csv",
        "test": "application_test.csv",
        "bureau": "bureau.csv",
        "bureau_balance": "bureau_balance.csv",
        "pos_cash": "POS_CASH_balance.csv",
        "credit_card": "credit_card_balance.csv",
        "previous": "previous_application.csv",
        "installments": "installments_payments.csv"
    }

    data = {}
    print("Memuat data...")
    for name, path in file_paths.items():
        try:
            # Gunakan dtype object=str untuk mencegah warning pada kolom ID
            data[name] = pd.read_csv(path, low_memory=False)
            print(f"  ✅ '{name}' dimuat: {data[name].shape[0]} baris.")
        except FileNotFoundError:
            print(f"  ❌ ERROR: File '{path}' tidak ditemukan. Lewati.")
    return data

In [33]:
# --- 2. Fungsi Agregasi Bureau dan Bureau Balance ---
def bureau_and_balance_agg(bureau, bureau_balance):
    """
    Melakukan agregasi bertingkat: bureau_balance -> bureau -> SK_ID_CURR.
    """
    print("\n[START] Feature Engineering: Bureau & Bureau Balance...")

    # Agregasi bureau_balance (bb) ke SK_ID_BUREAU
    bb_counts = bureau_balance.groupby('SK_ID_BUREAU')['STATUS'].value_counts(normalize=True).unstack(fill_value=0)
    bb_counts.columns = [f'BB_STATUS_{col}_RATIO' for col in bb_counts.columns]
    bb_agg = bureau_balance.groupby('SK_ID_BUREAU').agg(
        BB_MONTHS_COUNT=('MONTHS_BALANCE', 'count'),
        BB_MONTHS_MEAN=('MONTHS_BALANCE', 'mean')
    )
    bb_final = pd.merge(bb_counts, bb_agg, left_index=True, right_index=True, how='outer').reset_index()
    bureau_merged = pd.merge(bureau, bb_final, on='SK_ID_BUREAU', how='left')

    # Agregasi bureau_merged ke SK_ID_CURR
    bureau_agg = bureau_merged.groupby('SK_ID_CURR').agg({
        'SK_ID_BUREAU': 'count',
        'AMT_CREDIT_SUM': ['mean', 'max', 'sum'],
        'DAYS_CREDIT': ['mean', 'max'],
        'CREDIT_DAY_OVERDUE': ['mean', 'max'],
        'BB_MONTHS_COUNT': ['mean', 'sum']
    })

    bureau_agg.columns = [f'BUREAU_{"_".join(col).upper()}' for col in bureau_agg.columns]
    bureau_agg = bureau_agg.reset_index()

    # Gabungkan semua fitur kategorikal BB yang diagregasi
    bb_status_cols = [col for col in bb_final.columns if 'BB_STATUS' in col]
    for col in bb_status_cols:
        status_agg = bureau_merged.groupby('SK_ID_CURR')[col].agg(['mean']).reset_index()
        status_agg.columns = ['SK_ID_CURR', f'BUREAU_{col}_MEAN']
        bureau_agg = pd.merge(bureau_agg, status_agg, on='SK_ID_CURR', how='left')

    print("[END] Feature Engineering: Bureau & Bureau Balance selesai.")
    return bureau_agg

In [34]:
# --- 3. Fungsi Main Execution (Modifikasi untuk Integrasi XGBoost) ---
def run_full_pipeline():

    data = load_data()

    if 'train' not in data or 'test' not in data:
        print("\n[GAGAL] Data aplikasi utama (train/test) tidak lengkap.")
        return

    # Inisialisasi DataFrame akhir
    df_train_final = data['train'].copy()
    df_test_final = data['test'].copy()

    # --- PROSES A: Agregasi Tabel Pendukung ---
    all_features = {}

    # A. Bureau & Bureau Balance
    if 'bureau' in data and 'bureau_balance' in data:
        bureau_features = bureau_and_balance_agg(data['bureau'], data['bureau_balance'])
        all_features['bureau'] = bureau_features

    # --- PROSES B: Penggabungan (Merge) Akhir ke Data Aplikasi ---

    print("\n[START] Penggabungan Fitur ke Train dan Test...")

    for name, features_df in all_features.items():
        df_train_final = pd.merge(df_train_final, features_df, on='SK_ID_CURR', how='left')
        df_test_final = pd.merge(df_test_final, features_df, on='SK_ID_CURR', how='left')
        print(f"  ✅ Fitur dari '{name}' berhasil digabungkan.")

    # Hapus data mentah yang besar dari memori
    del data; gc.collect()

    print(f"\nDimensi Data Training Final: {df_train_final.shape}")
    print(f"Dimensi Data Testing Final: {df_test_final.shape}")

    # --- PROSES C: Persiapan Data untuk Model XGBoost ---

    print("\n--- Persiapan Data untuk Pemodelan XGBoost ---")

    # 1. Pemisahan Final Fitur (X) dan Target (y)

    # Simpan ID Pelanggan dari Test Set untuk file submission
    test_ids = df_test_final['SK_ID_CURR']

    # Hapus ID Pelanggan dan TARGET dari fitur training (X_train)
    X_train_full = df_train_final.drop(['TARGET', 'SK_ID_CURR'], axis=1)
    y_train = df_train_final['TARGET']

    # Hapus ID Pelanggan dari fitur testing (X_test)
    X_test_full = df_test_final.drop('SK_ID_CURR', axis=1)

    # 2. Identifikasi Tipe Kolom (Hanya menggunakan kolom yang ada di kedua set)

    # Isi NaN pada kolom kategorikal di train/test agar tidak gagal di OneHotEncoder
    X_train_full[X_train_full.select_dtypes(include='object').columns] = X_train_full.select_dtypes(include='object').fillna('Missing')
    X_test_full[X_test_full.select_dtypes(include='object').columns] = X_test_full.select_dtypes(include='object').fillna('Missing')

    numerical_cols = X_train_full.select_dtypes(include=np.number).columns.tolist()
    categorical_cols = X_train_full.select_dtypes(include='object').columns.tolist()

    # --- PROSES D: Pipelining Pra-pemrosesan & Model XGBoost ---

    # Pra-pemrosesan untuk kolom numerik
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    # Pra-pemrosesan untuk kolom kategorikal
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ],
        remainder='passthrough'
    )

    # Menghitung rasio kelas untuk scaling positif di XGBoost
    scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]

    # Model Pipeline Penuh
    xgb_model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', XGBClassifier(
            objective='binary:logistic',
            eval_metric='auc',
            n_estimators=1000,
            learning_rate=0.02,
            max_depth=4,
            scale_pos_weight=scale_pos_weight,
            n_jobs=-1,
            random_state=42,
            use_label_encoder=False # Harus False untuk versi XGBoost terbaru
        ))
    ])

    print("\n[START] Melatih Model XGBoost (1000 Trees)...")
    xgb_model.fit(X_train_full, y_train)
    print("[END] Pelatihan Model Selesai.")

    # --- PROSES E: Evaluasi dan Prediksi ---

    # Evaluasi pada Training Set
    y_train_pred_proba = xgb_model.predict_proba(X_train_full)[:, 1]
    train_auc = roc_auc_score(y_train, y_train_pred_proba)

    print(f"\n=============================================")
    print(f"🎉 Kinerja Model XGBoost (Training Set - AUC): {train_auc:.4f}")
    print("=============================================")

    # Prediksi pada Testing Set (Probabilitas)
    test_predictions_proba = xgb_model.predict_proba(X_test_full)[:, 1]

    # Pembuatan Submission File
    submission = pd.DataFrame({
        'SK_ID_CURR': test_ids,
        'TARGET': test_predictions_proba
    })

    submission.to_csv('xgboost_submission.csv', index=False)

    print("\n✅ Prediksi Data Test Selesai dan disimpan ke 'xgboost_submission.csv'")
    print("Head dari file submission:")
    print(submission.head())

# Eksekusi script utama
if __name__ == "__main__":
    run_full_pipeline()

Memuat data...
  ✅ 'train' dimuat: 307511 baris.
  ✅ 'test' dimuat: 48744 baris.
  ✅ 'bureau' dimuat: 1716428 baris.
  ✅ 'bureau_balance' dimuat: 27299925 baris.
  ✅ 'pos_cash' dimuat: 10001358 baris.
  ✅ 'credit_card' dimuat: 3840312 baris.
  ✅ 'previous' dimuat: 1670214 baris.
  ✅ 'installments' dimuat: 13605401 baris.

[START] Feature Engineering: Bureau & Bureau Balance...
[END] Feature Engineering: Bureau & Bureau Balance selesai.

[START] Penggabungan Fitur ke Train dan Test...
  ✅ Fitur dari 'bureau' berhasil digabungkan.

Dimensi Data Training Final: (307511, 140)
Dimensi Data Testing Final: (48744, 139)

--- Persiapan Data untuk Pemodelan XGBoost ---

[START] Melatih Model XGBoost (1000 Trees)...
[END] Pelatihan Model Selesai.

🎉 Kinerja Model XGBoost (Training Set - AUC): 0.7856

✅ Prediksi Data Test Selesai dan disimpan ke 'xgboost_submission.csv'
Head dari file submission:
   SK_ID_CURR    TARGET
0      100001  0.326874
1      100005  0.619582
2      100013  0.246138
3     